In [64]:
import numpy as np
import pandas as pd
import QuantLib as ql
from scipy.interpolate import interp1d
from scipy.stats import norm

In [65]:
strike_price = 110
spot_price = 100
risk_free_rate = 0.05 
volatility = 0.2
calculation_date = ql.Date(17,10,2021)

option_type = ql.Option.Call
maturity_date = ql.Date(17,12,2021)
dividend_rate = 0.0

day_count = ql.Actual365Fixed()
calendar = ql.UnitedStates(ql.UnitedStates.NYSE)
ql.Settings.instance().evaluationDate = calculation_date

payoff = ql.PlainVanillaPayoff(option_type, strike_price)
exercise = ql.EuropeanExercise(maturity_date)
european_option = ql.VanillaOption(payoff, exercise)

spot_handle = ql.QuoteHandle(ql.SimpleQuote(spot_price))
flat_ts = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, risk_free_rate, day_count))
dividend_yield = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, dividend_rate, day_count))
flat_vol_ts = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(calculation_date, calendar, volatility, day_count))
bsm_process = ql.BlackScholesMertonProcess(spot_handle, dividend_yield, flat_ts, flat_vol_ts)

european_option.setPricingEngine(ql.AnalyticEuropeanEngine(bsm_process))
bs_price = european_option.NPV()

delta = european_option.delta()
gamma = european_option.gamma()


In [66]:
bs_price

0.6291633939197441

In [67]:
delta

0.1532418203802124

In [68]:
gamma

0.02892516373607091

-------------------------------------------------------------------

In [69]:
data = pd.read_excel('Project 2 Tesla Data.xlsx',skiprows=3)
data.head()

,Unnamed: 0,US0003M Index,US0012M Index,USSW2 Curncy,USSW3 Curncy,USSW4 Curncy,USSW5 Curncy,USSW6 Curncy,USSW7 Curncy,USSW8 Curncy,USSW9 Curncy,USSW10 Curncy,USSW15 Curncy,USSW20 Curncy,USSW30 Curncy
0,Dates,PX_LAST,PX_LAST,PX_LAST,PX_LAST,PX_LAST,PX_LAST,PX_LAST,PX_LAST,PX_LAST,PX_LAST,PX_LAST,PX_LAST,PX_LAST,PX_LAST
1,2021-10-01 00:00:00,0.13313,0.23488,0.3713,0.6289,0.8476,1.0187,1.1561,1.2662,1.3515,1.4204,1.4798,1.6662,1.747,1.7687
2,2021-10-04 00:00:00,0.12663,0.232,0.3733,0.6336,0.8522,1.0245,1.1622,1.2736,1.3605,1.43,1.49,1.6775,1.7586,1.781
3,2021-10-05 00:00:00,0.124,0.23688,0.3864,0.6544,0.8766,1.051,1.1961,1.311,1.4007,1.473,1.5349,1.7266,1.8096,1.8335
4,2021-10-06 00:00:00,0.124,0.24113,0.3949,0.6649,0.8885,1.0619,1.2032,1.3155,1.4027,1.473,1.533,1.7195,1.8005,1.8227


In [70]:
file_path = 'Project 2 Tesla Data.xlsx'

try:
    data.head()
    swap_curve_df = pd.read_excel(file_path, sheet_name='Swap Curve', skiprows=4, index_col=0)
    stock_and_vol_data_df = pd.read_excel(file_path, sheet_name='Stock and Vol Data', skiprows=2, index_col=0)
except Exception as e:
    error_message = str(e)

# Check if the data was successfully read or not
if 'swap_curve_df' in locals() and 'stock_and_vol_data_df' in locals():
    # Successfully read the data
    success = True
    swap_curve_preview = swap_curve_df.head()  # Preview the first few rows
    stock_and_vol_data_preview = stock_and_vol_data_df.head()  # Preview the first few rows
else:
    # Failed to read the data
    success = False
    swap_curve_preview = error_message
    stock_and_vol_data_preview = error_message

(success, swap_curve_preview, stock_and_vol_data_preview)

(True,
             PX_LAST  PX_LAST.1  PX_LAST.2  PX_LAST.3  PX_LAST.4  PX_LAST.5  \
 Dates                                                                        
 2021-10-01  0.13313    0.23488     0.3713     0.6289     0.8476     1.0187   
 2021-10-04  0.12663    0.23200     0.3733     0.6336     0.8522     1.0245   
 2021-10-05  0.12400    0.23688     0.3864     0.6544     0.8766     1.0510   
 2021-10-06  0.12400    0.24113     0.3949     0.6649     0.8885     1.0619   
 2021-10-07  0.12363    0.24313     0.4125     0.6978     0.9276     1.1063   
 
             PX_LAST.6  PX_LAST.7  PX_LAST.8  PX_LAST.9  PX_LAST.10  \
 Dates                                                                
 2021-10-01     1.1561     1.2662     1.3515     1.4204      1.4798   
 2021-10-04     1.1622     1.2736     1.3605     1.4300      1.4900   
 2021-10-05     1.1961     1.3110     1.4007     1.4730      1.5349   
 2021-10-06     1.2032     1.3155     1.4027     1.4730      1.5330   
 2021-10-07 

In [71]:
# Ensure the index is in datetime format
stock_and_vol_data_df.index = pd.to_datetime(stock_and_vol_data_df.index)

for date, row in stock_and_vol_data_df.iterrows():
    # Convert the index (which is a pandas Timestamp) to a QuantLib Date
    ql_date = ql.Date(date.day, date.month, date.year)

    # Set the evaluation date in QuantLib
    ql.Settings.instance().evaluationDate = ql_date

In [72]:
stock_and_vol_data_df.head()

,PX_LAST,1M_CALL_IMP_VOL,1M_CALL_IMP_VOL.1,1M_CALL_IMP_VOL.2,1M_CALL_IMP_VOL.3,1M_CALL_IMP_VOL.4,1M_CALL_IMP_VOL.5,1M_CALL_IMP_VOL.6,2M_CALL_IMP_VOL,2M_CALL_IMP_VOL.1,2M_CALL_IMP_VOL.2,2M_CALL_IMP_VOL.3,2M_CALL_IMP_VOL.4,2M_CALL_IMP_VOL.5,2M_CALL_IMP_VOL.6
Dates,,,,,,,,,,,,,,,
2021-10-01,775.22,39.9621,40.3410,42.6079,45.3520,49.8035,62.1079,82.4243,41.9831,42.3314,44.7896,47.2430,50.9368,60.2199,78.4293
2021-10-01,781.53,42.0468,41.2154,42.9103,45.2499,49.1517,59.8281,80.9715,43.8222,43.6656,45.5394,47.7096,51.2537,60.0779,78.0672
2021-10-05,780.59,39.2900,39.4811,41.7489,44.5091,49.4001,63.4489,87.6114,44.2430,44.5035,46.7868,49.3566,52.9538,62.3818,81.6690
2021-10-06,782.75,38.4198,38.8774,41.0209,43.6354,48.5977,61.3141,83.2880,44.1528,44.3253,46.5429,48.8851,52.4051,61.4558,80.3056
2021-10-07,793.61,39.7719,38.3212,39.8688,42.2114,46.3740,58.9909,81.6114,44.1022,43.0259,44.7330,46.9329,50.0771,58.4750,76.9780


In [73]:
table1 = stock_and_vol_data_df.loc['2021-10-18':'2021-12-17']
table1.head()

,PX_LAST,1M_CALL_IMP_VOL,1M_CALL_IMP_VOL.1,1M_CALL_IMP_VOL.2,1M_CALL_IMP_VOL.3,1M_CALL_IMP_VOL.4,1M_CALL_IMP_VOL.5,1M_CALL_IMP_VOL.6,2M_CALL_IMP_VOL,2M_CALL_IMP_VOL.1,2M_CALL_IMP_VOL.2,2M_CALL_IMP_VOL.3,2M_CALL_IMP_VOL.4,2M_CALL_IMP_VOL.5,2M_CALL_IMP_VOL.6
Dates,,,,,,,,,,,,,,,
2021-10-18,870.11,39.0968,35.0381,34.6969,35.5223,37.2086,42.7095,57.5567,38.1697,34.6860,34.4770,35.2923,36.9576,42.2516,55.7259
2021-10-19,864.27,38.9873,35.5647,35.8108,36.9335,39.1516,45.1975,59.6323,37.9855,34.8035,34.8771,35.8662,37.8256,43.2998,56.4778
2021-10-20,865.80,37.8829,34.8072,34.9670,36.1522,37.9943,43.1774,56.6007,37.0735,34.4335,34.5484,35.5587,37.4108,42.4941,54.8838
2021-10-21,894.00,39.4596,35.3877,35.1048,35.9636,37.6684,43.0527,58.2194,38.9553,35.6205,35.5761,36.2682,37.8056,42.5385,55.0598
2021-10-22,909.68,39.7986,35.8091,35.4431,36.1680,37.6649,42.5688,56.9291,39.5296,36.1163,36.0206,36.8009,38.3566,42.8892,55.1847


In [74]:
# Ensure the index is in datetime format
stock_and_vol_data_df.index = pd.to_datetime(swap_curve_df.index)

for date, row in swap_curve_df.iterrows():
    # Convert the index (which is a pandas Timestamp) to a QuantLib Date
    ql_date = ql.Date(date.day, date.month, date.year)

    # Set the evaluation date in QuantLib
    ql.Settings.instance().evaluationDate = ql_date

In [75]:
data2 = swap_curve_df.loc['2021-10-18':'2021-12-17']
r = data2['PX_LAST'].values

In [76]:
# fixed Strike price of 110% OTM call option
strike = table1['PX_LAST'].iloc[0] * 1.1

In [77]:
# create an option table
option_table = pd.DataFrame(index=table1.index)
option_table['price'] = table1['PX_LAST']
option_table['strike'] = strike
option_table['r'] = r

# Update 'implied_vol' in 'option_table' with '1M_CALL_IMP_VOL.2' values from 'table1' for the date range 10-18 to 11-22
option_table.loc['2021-10-18':'2021-11-22', 'implied_vol'] = table1.loc['2021-10-18':'2021-11-22', '2M_CALL_IMP_VOL.2']

# Update 'implied_vol' in 'option_table' with '2M_CALL_IMP_VOL.2' values from 'table1' for the date range 11-23 to 12-17
option_table.loc['2021-11-23':'2021-12-17', 'implied_vol'] = table1.loc['2021-11-23':'2021-12-17', '1M_CALL_IMP_VOL.2']

option_table.head()

,price,strike,r,implied_vol
Dates,,,,
2021-10-18,870.11,957.121,0.13150,34.4770
2021-10-19,864.27,957.121,0.12950,34.8771
2021-10-20,865.80,957.121,0.12825,34.5484
2021-10-21,894.00,957.121,0.12388,35.5761
2021-10-22,909.68,957.121,0.12488,36.0206


In [78]:
# create a few zero columns to store results in option_table
option_table['option_price'] = 0.0
option_table['option_pnl'] = 0.0
option_table['delta'] = 0.0
option_table['delta_pnl'] = 0.0
option_table['gamma'] = 0.0
option_table['gamma_pnl'] = 0.0
option_table['rho'] = 0.0
option_table['rho_pnl'] = 0.0
option_table['vega'] = 0.0
option_table['vega_pnl'] = 0.0
option_table['theta'] = 0.0
option_table['theta_pnl'] = 0.0

In [79]:
volatility = table1['2M_CALL_IMP_VOL.2'].iloc[0]/100
spot_price = option_table['price'].iloc[0]
strike_price = option_table['strike'].iloc[0]
risk_free_rate = option_table['r'].iloc[0]

In [80]:
calculation_date = ql.Date(18,10,2021)

option_type = ql.Option.Call
maturity_date = ql.Date(17,12,2021)
dividend_rate = 0.0

day_count = ql.Actual365Fixed()
calendar = ql.UnitedStates(ql.UnitedStates.NYSE)
ql.Settings.instance().evaluationDate = calculation_date

payoff = ql.PlainVanillaPayoff(option_type, strike_price)
exercise = ql.EuropeanExercise(maturity_date)
european_option = ql.VanillaOption(payoff, exercise)

spot_handle = ql.QuoteHandle(ql.SimpleQuote(spot_price))
flat_ts = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, risk_free_rate, day_count))
dividend_yield = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, dividend_rate, day_count))
flat_vol_ts = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(calculation_date, calendar, volatility, day_count))
bsm_process = ql.BlackScholesMertonProcess(spot_handle, dividend_yield, flat_ts, flat_vol_ts)

european_option.setPricingEngine(ql.AnalyticEuropeanEngine(bsm_process))
bs_price = european_option.NPV()

delta = european_option.delta()
gamma = european_option.gamma()
vega = european_option.vega()
theta = european_option.theta()
rho = european_option.rho()

In [81]:
# Calculate the time to maturity (T)
T = day_count.yearFraction(calculation_date, maturity_date)
T

0.1643835616438356

In [82]:
option_table['option_price'].iloc[0] = bs_price
option_table['delta'].iloc[0] = delta
option_table['gamma'].iloc[0] = gamma
option_table['vega'].iloc[0] = vega
option_table['theta'].iloc[0] = theta
option_table['rho'].iloc[0] = rho
option_table.head()

C:\Users\zhong\AppData\Local\Temp\ipykernel_21104\1674634707.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  option_table['option_price'].iloc[0] = bs_price
C:\Users\zhong\AppData\Local\Temp\ipykernel_21104\1674634707.py:2: FutureWarning

,price,strike,r,implied_vol,option_price,option_pnl,delta,delta_pnl,gamma,gamma_pnl,rho,rho_pnl,vega,vega_pnl,theta,theta_pnl
Dates,,,,,,,,,,,,,,,,
2021-10-18,870.11,957.121,0.13150,34.4770,23.887783,0.0,0.323726,0.0,0.002954,0.0,42.376407,0.0,126.765816,0.0,-166.835555,0.0
2021-10-19,864.27,957.121,0.12950,34.8771,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
2021-10-20,865.80,957.121,0.12825,34.5484,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
2021-10-21,894.00,957.121,0.12388,35.5761,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
2021-10-22,909.68,957.121,0.12488,36.0206,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0


In [83]:
# define a function to calculate the option price and greeks
def update_bsm_process(date, spot_price, risk_free_rate, volatility):
    calculation_date = ql.Date(date.day, date.month, date.year)
    ql.Settings.instance().evaluationDate = calculation_date
    spot_handle = ql.QuoteHandle(ql.SimpleQuote(spot_price))
    flat_ts = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, risk_free_rate, day_count))
    flat_vol_ts = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(calculation_date, calendar, volatility, day_count))
    bsm_process = ql.BlackScholesMertonProcess(spot_handle, dividend_yield, flat_ts, flat_vol_ts)
    return bsm_process

# Loop through the rows of the option_table and calculate the option price and greeks
for index, row in option_table.iterrows():
    bsm_process = update_bsm_process(index, row['price'], row['r'], row['implied_vol']/100)
    european_option.setPricingEngine(ql.AnalyticEuropeanEngine(bsm_process))
    bs_price = european_option.NPV()
    delta = european_option.delta()
    gamma = european_option.gamma()
    vega = european_option.vega()
    theta = european_option.theta()
    rho = european_option.rho()
    option_table.at[index, 'option_price'] = bs_price
    option_table.at[index, 'delta'] = delta 
    option_table.at[index, 'gamma'] = gamma
    option_table.at[index, 'vega'] = vega
    option_table.at[index, 'theta'] = theta
    option_table.at[index, 'rho'] = rho
option_table.head()

,price,strike,r,implied_vol,option_price,option_pnl,delta,delta_pnl,gamma,gamma_pnl,rho,rho_pnl,vega,vega_pnl,theta,theta_pnl
Dates,,,,,,,,,,,,,,,,
2021-10-18,870.11,957.121,0.13150,34.4770,23.887783,0.0,0.323726,0.0,0.002954,0.0,42.376407,0.0,126.765816,0.0,-166.835555,0.0
2021-10-19,864.27,957.121,0.12950,34.8771,22.014617,0.0,0.305609,0.0,0.002893,0.0,39.136251,0.0,121.822397,0.0,-162.778889,0.0
2021-10-20,865.80,957.121,0.12825,34.5484,21.588685,0.0,0.304578,0.0,0.002936,0.0,38.473040,0.0,120.817980,0.0,-162.390431,0.0
2021-10-21,894.00,957.121,0.12388,35.5761,32.030650,0.0,0.390746,0.0,0.003054,0.0,49.550326,0.0,135.622518,0.0,-193.788719,0.0
2021-10-22,909.68,957.121,0.12488,36.0206,38.647774,0.0,0.438816,0.0,0.003072,0.0,55.314873,0.0,140.474771,0.0,-209.924855,0.0


In [84]:
# define a function to calculate T
def calculate_T(date, maturity_date):
    calculation_date = ql.Date(date.day, date.month, date.year)
    ql.Settings.instance().evaluationDate = calculation_date
    T = day_count.yearFraction(calculation_date, maturity_date)
    return T

# Loop through the rows of the option_table and calculate the time to maturity (T)
for index, row in option_table.iterrows(): 
    T = calculate_T(index, maturity_date)
    option_table.at[index, 'T'] = T

In [85]:
option_table['OptQuantity'] = 1000
option_table['StockQuantity'] = -option_table['delta'] * 1000
option_table.head()


# Calculate the change in price as the current price minus the previous price
option_table['price_change'] = option_table['price'] - option_table['price'].shift(1)



# Delta PnL = Delta * Price Change
option_table['delta_pnl'] = option_table['delta'] * option_table['price_change']

# Gamma PnL = 0.5 * Gamma * (Price Change)^2
option_table['gamma_pnl'] = 0.5 * option_table['gamma'] * (option_table['price_change'] ** 2)

# Vega PnL = Vega * (Implied Vol Change)
option_table['vega_pnl'] = option_table['vega'] * (option_table['implied_vol'] - option_table['implied_vol'].shift(1))

# Theta PnL = Theta * Time Change
option_table['theta_pnl'] = option_table['theta'] * (option_table['T'] - option_table['T'].shift(1))

# Rho PnL = Rho * (Interest Rate Change)
option_table['rho_pnl'] = option_table['rho'] * (option_table['r'] - option_table['r'].shift(1))

option_table['option_pnl'] = option_table['option_price'] - option_table['option_price'].shift(1)
# option_table['option_pnl'] = option_table['delta_pnl'] + option_table['gamma_pnl'] + option_table['vega_pnl'] + option_table['theta_pnl'] + option_table['rho_pnl']

option_table['stock_pnl'] = option_table['price_change']
option_table['total_pnl'] = option_table['option_pnl'] + option_table['stock_pnl']

option_table.head()

,price,strike,r,implied_vol,option_price,option_pnl,delta,delta_pnl,gamma,gamma_pnl,...,vega,vega_pnl,theta,theta_pnl,T,OptQuantity,StockQuantity,price_change,stock_pnl,total_pnl
Dates,,,,,,,,,,,,,,,,,,,,,
2021-10-18,870.11,957.121,0.13150,34.4770,23.887783,NaN,0.323726,NaN,0.002954,NaN,...,126.765816,NaN,-166.835555,NaN,0.164384,1000,-323.726417,NaN,NaN,NaN
2021-10-19,864.27,957.121,0.12950,34.8771,22.014617,-1.873166,0.305609,-1.784757,0.002893,0.049332,...,121.822397,48.741141,-162.778889,0.445970,0.161644,1000,-305.609023,-5.84,-5.84,-7.713166
2021-10-20,865.80,957.121,0.12825,34.5484,21.588685,-0.425932,0.304578,0.466004,0.002936,0.003436,...,120.817980,-39.712870,-162.390431,0.444905,0.158904,1000,-304.577851,1.53,1.53,1.104068
2021-10-21,894.00,957.121,0.12388,35.5761,32.030650,10.441965,0.390746,11.019027,0.003054,1.214464,...,135.622518,139.379262,-193.788719,0.530928,0.156164,1000,-390.745635,28.20,28.20,38.641965
2021-10-22,909.68,957.121,0.12488,36.0206,38.647774,6.617124,0.438816,6.880636,0.003072,0.377604,...,140.474771,62.441036,-209.924855,0.575137,0.153425,1000,-438.816084,15.68,15.68,22.297124


In [86]:
output_table = option_table[['OptQuantity', 'option_price', 'StockQuantity', 'price', 'implied_vol', 'delta', 'delta_pnl','gamma', 'gamma_pnl', 'vega','vega_pnl', 'theta', 'theta_pnl', 'rho', 'rho_pnl','option_pnl', 'stock_pnl', 'total_pnl']]
output_table.head()

,OptQuantity,option_price,StockQuantity,price,implied_vol,delta,delta_pnl,gamma,gamma_pnl,vega,vega_pnl,theta,theta_pnl,rho,rho_pnl,option_pnl,stock_pnl,total_pnl
Dates,,,,,,,,,,,,,,,,,,
2021-10-18,1000,23.887783,-323.726417,870.11,34.4770,0.323726,NaN,0.002954,NaN,126.765816,NaN,-166.835555,NaN,42.376407,NaN,NaN,NaN,NaN
2021-10-19,1000,22.014617,-305.609023,864.27,34.8771,0.305609,-1.784757,0.002893,0.049332,121.822397,48.741141,-162.778889,0.445970,39.136251,-0.078273,-1.873166,-5.84,-7.713166
2021-10-20,1000,21.588685,-304.577851,865.80,34.5484,0.304578,0.466004,0.002936,0.003436,120.817980,-39.712870,-162.390431,0.444905,38.473040,-0.048091,-0.425932,1.53,1.104068
2021-10-21,1000,32.030650,-390.745635,894.00,35.5761,0.390746,11.019027,0.003054,1.214464,135.622518,139.379262,-193.788719,0.530928,49.550326,-0.216535,10.441965,28.20,38.641965
2021-10-22,1000,38.647774,-438.816084,909.68,36.0206,0.438816,6.880636,0.003072,0.377604,140.474771,62.441036,-209.924855,0.575137,55.314873,0.055315,6.617124,15.68,22.297124


In [87]:
output_table.to_excel('output_table.xlsx')

In [88]:
option_table['log_returns'] = np.log(option_table['price'] / option_table['price'].shift(1))
mean_return = option_table['log_returns'].mean()    
# Calculate the sample variance (unbiased estimator) of log returns
sample_variance = option_table['log_returns'].var(ddof=1)
option_table['daily_actual_variance'] = sample_variance

# daily_volatility = np.sqrt(sample_variance)
# option_table['annualized_actual_volatility'] = daily_volatility * np.sqrt(260)
option_table['realized_variance'] = sample_variance * 260

option_table['implied_variance'] = option_table['implied_vol'] ** 2

In [89]:
option_table.index = pd.to_datetime(option_table.index)
dt_series = option_table.index.to_series().diff().dt.days / 365
option_table['dPi'] = -0.5 * (option_table['realized_variance'] - option_table['implied_variance']) * (option_table['price']**2) * option_table['gamma'] * dt_series
total_dPi = option_table['dPi'].sum()
print(total_dPi)

681498.1901094485
